### RunnableParallel


In [4]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

In [13]:
chat_template_books = ChatPromptTemplate.from_template(
    """
    Suggest three of the best intermediate level {programming language} books.
    Answer only by listing the books.
    """
)

chat_template_projects = ChatPromptTemplate.from_template(
    """
    Suggest three intesting {programming language} projects for a developer to work on.
    Answer only by listing the projects.
    """
)

chat_template_time = ChatPromptTemplate.from_template(
    """
    I am an intermediate level programmer.
    Consider the following literature:
    {books}
    Also, consider these projects:
    {projects}
    Based on these, how much time would it take to complete the literature and the projects?
    """
)

In [14]:
chat = ChatOpenAI(
    api_key=api_key,
    model_name="gpt-4.1-mini",
    temperature=0,
    max_tokens=500,
    seed=42,
)

In [15]:
string_parser = StrOutputParser()

In [16]:
chain_book = chat_template_books | chat | string_parser
chain_project = chat_template_projects | chat | string_parser

In [17]:
chain_parallel = RunnableParallel(
    chains={"books": chain_book, "projects": chain_project}
)

In [18]:
chain_parallel.invoke({"programming language": "Python"})

{'chains': {'books': '1. "Fluent Python" by Luciano Ramalho  \n2. "Effective Python: 90 Specific Ways to Write Better Python" by Brett Slatkin  \n3. "Python Cookbook" by David Beazley and Brian K. Jones',
  'projects': '1. Personal Finance Tracker with Data Visualization  \n2. AI-Powered Chatbot for Customer Support  \n3. Automated Web Scraper and Data Analyzer for Market Trends'}}

In [12]:
chain_parallel.get_graph().print_ascii()

                +-----------------------+                  
                | Parallel<chains>Input |                  
                +-----------------------+                  
                             *                             
                             *                             
                             *                             
            +-------------------------------+              
            | Parallel<books,projects>Input |              
            +-------------------------------+              
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                            

In [26]:
chain_time1 = (
    RunnableParallel({"books": chain_book, "projects": chain_project})
    | chat_template_time
    | chat
    | string_parser
)

In [27]:
chain_time2 = (
    {"books": chain_book, "projects": chain_project}
    | chat_template_time
    | chat
    | string_parser
)

In [28]:
print(chain_time2.invoke({"programming language": "Python"}))

Great question! Estimating the time to complete both the literature and projects depends on several factors such as your current skill level, available daily time, learning speed, and project complexity. Since you mentioned you are an intermediate programmer, I'll provide a rough estimate based on typical pacing.

---

### Literature Time Estimates

1. **"Fluent Python" by Luciano Ramalho**  
   - Length: ~770 pages  
   - Content: Deep dive into Pythonic idioms, advanced features, and best practices  
   - Estimated time: 6-8 weeks (assuming 1-1.5 hours/day)  
   - Notes: This book is dense and requires hands-on practice to absorb concepts.

2. **"Effective Python" by Brett Slatkin**  
   - Length: ~256 pages  
   - Content: 90 actionable tips for writing better Python code  
   - Estimated time: 3-4 weeks (assuming 1 hour/day)  
   - Notes: Can be read in smaller chunks; good for daily practice.

3. **"Python Cookbook" by David Beazley and Brian K. Jones**  
   - Length: ~700 pages  

In [29]:
chain_time2.get_graph().print_ascii()

            +-------------------------------+              
            | Parallel<books,projects>Input |              
            +-------------------------------+              
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOpenAI |                      | ChatOpenAI |     
    +------------+                      +------------+     
           *                                   *           
           *                            